In [1]:
#Librerias utilizadas
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import datetime
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt
nltk.download("punk")
nltk.download("stopwords")

[nltk_data] Error loading punk: Package 'punk' not found in index
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jogabell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Lectura de datos de la plataforma HIPEGIve

In [2]:
data_names = pd.read_csv('data/names_tokenized_and_cleaned.csv')

In [3]:
names_hipe = data_names[['0','1','2','3','4']]

In [4]:
data = names_hipe.values.tolist()

In [5]:
def get_clean_from_nan(name_list):
    clean_names=[]
    for e in name_list:
        clean_name=[]
        for i in e:            
            if not ((type(i)==float) or (type(i)==type(None))):
                clean_name.append(i)
        clean_names.append(clean_name)
    return clean_names

In [6]:
clean_names = get_clean_from_nan(data)

In [7]:
len(clean_names)

11926

#### Lectura y limpieza de base de datos de apellidos hispanos 

In [8]:
surname_db = pd.read_csv('data/hispanic_surnames.csv')

In [9]:
def get_in_lower(lista):
    lower_surnames=[]
    for surname in lista:
        # Converting to Lowercase
        lower_surname = surname.lower()
        lower_surnames.append(lower_surname)
    return lower_surnames

In [10]:
lower_his_surnames = get_in_lower(surname_db['hispanic_surnames'].values.tolist())

### Inicio de comparación de apellidos

clean_names: es la lista de nombres dentro de la plataforma. 

lower_his_surnames: es la lista de apellidos hispanos más populares en USA.

In [11]:
def get_list_of_vectorized_surname(lista):    
#def get_dictionary_of_names_with_hispanic_surnames(lista):    
    list_of_vectors=[]
    for i_name in range(0,len(lista)):
        vector_list=[]
        for word in lista[i_name]:
            if word in lower_his_surnames:
                vector_list.append(1)
            else: 
                vector_list.append(0)
        list_of_vectors.append(vector_list)
    return list_of_vectors

In [12]:
vect_of_names = get_list_of_vectorized_surname(clean_names)

In [13]:
def get_index_for_hispanic_surname(vect_of_names):
    index_true_hisp=[]
    index_false_hisp=[]
    for vect_name in vect_of_names:
        total = sum(vect_name)
        if total > 0:
            index_true_hisp.append(True)
            index_false_hisp.append(False)
        else:
            index_true_hisp.append(False)
            index_false_hisp.append(True)
    return (pd.DataFrame(index_true_hisp),pd.DataFrame(index_false_hisp))

In [14]:
index_surnames = get_index_for_hispanic_surname(vect_of_names)

In [15]:
#Transformadno lista de nombres en la plataforma a dataframe
clean_names_pd = pd.DataFrame(clean_names)
# resultados de indicios de descendencia hispana
hispanic_results = clean_names_pd[index_surnames[0].values]
# resultados de indicios de descendencia no hispana
no_hispanic_results = clean_names_pd[index_surnames[1].values]

In [16]:
hispanic_results_cleaned = get_clean_from_nan(hispanic_results.values.tolist())
no_hispanic_results_cleaned = get_clean_from_nan(no_hispanic_results.values.tolist())

In [17]:
len(hispanic_results_cleaned)

5094

In [18]:
len(no_hispanic_results_cleaned)

6832

In [19]:
hispanic_df = pd.DataFrame(hispanic_results_cleaned)
no_hispanic_df = pd.DataFrame(no_hispanic_results_cleaned)
hispanic_df.to_csv('data/results_hispanic_donors.csv')
no_hispanic_df.to_csv('data/results_no_hispanic_donors.csv')

In [20]:
total=float(len(hispanic_results_cleaned)+len(no_hispanic_results_cleaned))
100.0*float(len(hispanic_results_cleaned))/total,100.0*float(len(no_hispanic_results_cleaned))/total


(42.71339929565655, 57.28660070434345)

### Restultados de descendencia.

Los valores en porcentaje que se obtienen para la participación hispana y no hispana en las donaciónes corresopnde a un 
42.7% de participación con ascendencia hispana-latina y un 57.3% no hispana. 